In [1]:
from bs4 import BeautifulSoup
import requests, pandas as pd
import urllib.request

link_to_crawl = "https://play.google.com/store/apps/details?id=vn.com.fsoft.myfsoft&hl=vi&gl=US"
res = requests.get(link_to_crawl)

content = BeautifulSoup(res.content, "html.parser")

In [2]:
user_divs = content.find_all("div", {"class": "X5PpBb"})
user_names = [div.text for div in user_divs]
user_names

['Hoài Phong', 'Thuy Nguyen', 'Mình Quân Nguyễn']

In [3]:
cmt_divs = content.find_all("div", {"class": "h3YV2d"})
comments = [div.text for div in cmt_divs]
comments

['Sao k viết đx @ vậy ạ', 'Ok', 'Ok']

Tuy nhiên, để đọc hết bài đánh giá này (vì có phần ẩn cần bấm vào showmore để hiển thị thêm) nên ta sẽ có 2 giải pháp!
- Download toàn bộ trang nguồn html từ website, lưu về html rồi thực hiện ETL với dữ liệu loại html
- Sử dụng selenium

## 1. Hãy làm quen với html

In [4]:
import codecs

f = codecs.open('myFPT_GooglePlay.html', 'r', 'utf-8')
full_content = BeautifulSoup(f.read())

In [5]:
user_divs = full_content.find_all("div", {"class": "X5PpBb"})
user_names = [div.text for div in user_divs]
print(f"First 5 user-names: \n {user_names[:5]} \n Total rating-results: {len(user_names)}")

First 5 user-names: 
 ['Hoài Phong', 'Thuy Nguyen', 'Mình Quân Nguyễn', 'Hoài Phong', 'Thuy Nguyen'] 
 Total rating-results: 560


In [6]:
cmt_divs = full_content.find_all("div", {"class": "h3YV2d"})
comments = [div.text for div in cmt_divs]

print(f"First 5 user-comments: \n {comments[:5]} \n Total rating-results: {len(comments)}")

First 5 user-comments: 
 ['Sao k viết đx @ vậy ạ', 'Ok', 'Ok', 'Sao k viết đx @ vậy ạ', 'Ok'] 
 Total rating-results: 560


In [7]:
rating_divs = full_content.find_all("div", {"class": "iXRFPc", "role": "img"})
rate_res = [div["aria-label"] for div in rating_divs]

print(f"First 5 rating results: \n {rate_res[:5]} \n Total rating-results: {len(rate_res)}")

First 5 rating results: 
 ['Được xếp hạng 3 sao/5 sao', 'Được xếp hạng 5 sao/5 sao', 'Được xếp hạng 5 sao/5 sao', 'Được xếp hạng 3 sao/5 sao', 'Được xếp hạng 5 sao/5 sao'] 
 Total rating-results: 560


In [8]:
crawl_df = pd.DataFrame({'user_name': user_names, 'comments': comments, 'ratings': rate_res})
crawl_df.head()

,user_name,comments,ratings
0,Hoài Phong,Sao k viết đx @ vậy ạ,Được xếp hạng 3 sao/5 sao
1,Thuy Nguyen,Ok,Được xếp hạng 5 sao/5 sao
2,Mình Quân Nguyễn,Ok,Được xếp hạng 5 sao/5 sao
3,Hoài Phong,Sao k viết đx @ vậy ạ,Được xếp hạng 3 sao/5 sao
4,Thuy Nguyen,Ok,Được xếp hạng 5 sao/5 sao


## 2. Using selenium

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

options = Options()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [10]:
driver.get(link_to_crawl)

In [11]:
import warnings

warnings.filterwarnings("ignore")

init_users = [x.text for x in driver.find_elements_by_class_name("X5PpBb")]
init_comments = [x.text for x in driver.find_elements_by_class_name("h3YV2d")]

In [12]:
selen_init_content = BeautifulSoup(driver.page_source)
rating_divs = selen_init_content.find_all("div", {"class": "iXRFPc", "role": "img"})
rate_res = [div["aria-label"] for div in rating_divs]

print(f"First 5 rating results: \n {rate_res[:5]} \n Total rating-results: {len(rate_res)}")

First 5 rating results: 
 ['Được xếp hạng 3 sao/5 sao', 'Được xếp hạng 5 sao/5 sao', 'Được xếp hạng 5 sao/5 sao'] 
 Total rating-results: 3


Tuy nhiên, ta chỉ view được 3 đối tượng đầu tiên,

In [15]:
button = '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/div[2]/div/div[1]/c-wiz[4]/section/div/div/div[5]/div/div/button/span'
new_web_dri = driver.find_element(by=By.XPATH, value=button)
new_web_dri.click()

### Auto crawl

In [32]:
selen_init_content = BeautifulSoup(driver.page_source)
rating_divs = selen_init_content.find_all("div", {"class": "iXRFPc", "role": "img"})
rate_res = [div["aria-label"] for div in rating_divs]

print(f"First 5 rating results: \n {rate_res[:5]} \n Total rating-results: {len(rate_res)}")

First 5 rating results: 
 ['Được xếp hạng 3 sao/5 sao', 'Được xếp hạng 5 sao/5 sao', 'Được xếp hạng 5 sao/5 sao', 'Được xếp hạng 3 sao/5 sao', 'Được xếp hạng 5 sao/5 sao'] 
 Total rating-results: 123
